In [ ]:
import os
import json
import logging
import sys
from datetime import datetime
from typing import Dict, Any, List, Optional
from dataclasses import dataclass, asdict
from collections import defaultdict
import uuid
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from tenacity import retry, stop_after_attempt, wait_fixed






In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [ ]:
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": "",
            "base_url": "",
            "api_type": "azure",
            "api_version": "2024-02-01"
        }
    ],
    "temperature": 0.7,
    "timeout": 120,
    "cache_seed": 42
}

In [ ]:
# Create directories
os.makedirs("data/conversation_history", exist_ok=True)
os.makedirs("data/progress_data", exist_ok=True)


In [ ]:

# Agent Configuration
class AgentConfig:
    @staticmethod
    def get_tutor_system_message() -> str:
        return """You are an expert Educational Tutor Agent. Explain concepts clearly for high-school students, provide step-by-step breakdowns, examples, and practice questions. Evaluate student responses, offer constructive feedback, and maintain a supportive tone. Use visual aids for visual learners when specified."""

    @staticmethod
    def get_student_system_message() -> str:
        return """You are a high-school Student Agent. Engage actively by:
- Attempting to solve problems and showing your steps (e.g., 'I factored x^2 - 7x + 10 as (x-5)(x-2)').
- Asking clarifying questions if unsure (e.g., 'Can you explain the quadratic formula again?').
- Reflecting on your learning (e.g., 'I understand factoring but struggle with graphing').
- Responding to tutor prompts with effort, even if unsure (e.g., 'I’ll try: ...').
If asked for progress, summarize recent activities (e.g., 'I worked on factoring quadratic equations')."""

    @staticmethod
    def get_progress_tracker_system_message() -> str:
        return """You are a Progress Tracker Agent. Generate student-friendly JSON progress reports using ProgressMemory data. Include a summary, strengths (skill_level > 0.7), areas for improvement (skill_level < 0.6), and recommendations. Handle minimal data gracefully."""

In [ ]:


# Conversation Memory (LangChain)
class LangChainConversationMemory:
    def __init__(self, session_id: str, storage_path: str = "data/conversation_history"):
        self.session_id = session_id
        self.storage_path = storage_path
        self.memory = ConversationBufferMemory(return_messages=True)
        self.metadata = {
            "session_id": session_id,
            "created_at": datetime.now().isoformat(),
            "subjects": [],
            "topics": []
        }
        os.makedirs(self.storage_path, exist_ok=True)
        self._load()

    def add_message(self, role: str, content: str, metadata: Optional[Dict[str, str]] = None):
        if role == "student":
            self.memory.chat_memory.add_message(HumanMessage(content=content))
        else:
            self.memory.chat_memory.add_message(AIMessage(content=content))
        if metadata:
            if "subject" in metadata and metadata["subject"] not in self.metadata["subjects"]:
                self.metadata["subjects"].append(metadata["subject"])
            if "topic" in metadata and metadata["topic"] not in self.metadata["topics"]:
                self.metadata["topics"].append(metadata["topic"])
        self._save()

    def get_messages(self) -> List[Dict[str, Any]]:
        return [
            {"role": "human" if isinstance(msg, HumanMessage) else "assistant", "content": msg.content}
            for msg in self.memory.chat_memory.messages
        ]

    def get_context(self) -> str:
        return self.memory.buffer

    def _save(self):
        filepath = os.path.join(self.storage_path, f"{self.session_id}.json")
        try:
            data = {
                "metadata": self.metadata,
                "messages": [
                    {"role": "human" if isinstance(msg, HumanMessage) else "assistant", "content": msg.content}
                    for msg in self.memory.chat_memory.messages
                ]
            }
            with open(filepath, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2)
        except Exception as e:
            logger.error(f"Failed to save conversation: {e}")

    def _load(self):
        filepath = os.path.join(self.storage_path, f"{self.session_id}.json")
        if os.path.exists(filepath):
            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    data = json.load(f)
                self.metadata = data.get("metadata", self.metadata)
                for msg in data.get("messages", []):
                    if msg["role"] == "human":
                        self.memory.chat_memory.add_message(HumanMessage(content=msg["content"]))
                    else:
                        self.memory.chat_memory.add_message(AIMessage(content=msg["content"]))
            except Exception as e:
                logger.error(f"Failed to load conversation: {e}")


In [ ]:

# Progress Memory
@dataclass
class LearningProgress:
    subject: str
    topic: str
    skill_level: float
    attempts: int
    successful_attempts: int

@dataclass
class LearningSession:
    session_id: str
    start_time: str
    subjects_covered: List[str]
    questions_asked: int
    questions_answered_correctly: int

class ProgressMemory:
    def __init__(self, student_id: str, storage_path: str = "data/progress_data"):
        self.student_id = student_id
        self.storage_path = storage_path
        self.progress_data: Dict[str, Dict[str, LearningProgress]] = defaultdict(dict)
        self.learning_sessions: List[LearningSession] = []
        os.makedirs(self.storage_path, exist_ok=True)
        self._load()

    def start_session(self, session_id: str) -> LearningSession:
        session = LearningSession(
            session_id=session_id,
            start_time=datetime.now().isoformat(),
            subjects_covered=[],
            questions_asked=0,
            questions_answered_correctly=0
        )
        self.learning_sessions.append(session)
        self._save()
        return session

    def update_progress(self, subject: str, topic: str, performance_score: float, was_successful: bool):
        if subject not in self.progress_data:
            self.progress_data[subject] = {}
        if topic not in self.progress_data[subject]:
            self.progress_data[subject][topic] = LearningProgress(subject, topic, 0.0, 0, 0)
        progress = self.progress_data[subject][topic]
        progress.attempts += 1
        if was_successful:
            progress.successful_attempts += 1
        progress.skill_level = (progress.skill_level * 0.7) + (performance_score * 0.3)
        logger.info(f"Updated progress: {subject}/{topic}, skill={progress.skill_level}")
        self._save()

    def update_session(self, session_id: str, subject: str, topic: str, question_asked: bool = False, correct_answer: bool = False):
        for session in self.learning_sessions:
            if session.session_id == session_id:
                if subject not in session.subjects_covered:
                    session.subjects_covered.append(subject)
                if question_asked:
                    session.questions_asked += 1
                if correct_answer:
                    session.questions_answered_correctly += 1
                logger.info(f"Updated session {session_id}: subject={subject}, questions_asked={session.questions_asked}")
                self._save()
                break

    def get_progress_report(self) -> Dict[str, Any]:
        return {
            "student_id": self.student_id,
            "subjects": {s: {t: asdict(p) for t, p in topics.items()} for s, topics in self.progress_data.items()},
            "sessions": [asdict(s) for s in self.learning_sessions],
            "timestamp": datetime.now().isoformat()
        }

    def _save(self):
        filepath = os.path.join(self.storage_path, f"{self.student_id}_progress.json")
        try:
            data = {
                "student_id": self.student_id,
                "progress_data": {
                    s: {t: asdict(p) for t, p in topics.items()}
                    for s, topics in self.progress_data.items()
                },
                "learning_sessions": [asdict(s) for s in self.learning_sessions]
            }
            with open(filepath, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2)
        except Exception as e:
            logger.error(f"Failed to save progress: {e}")

    def _load(self):
        filepath = os.path.join(self.storage_path, f"{self.student_id}_progress.json")
        if os.path.exists(filepath):
            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    data = json.load(f)
                for subject, topics in data.get("progress_data", {}).items():
                    self.progress_data[subject] = {
                        topic: LearningProgress(**progress)
                        for topic, progress in topics.items()
                    }
                self.learning_sessions = [
                    LearningSession(**session)
                    for session in data.get("learning_sessions", [])
                ]
            except Exception as e:
                logger.error(f"Failed to load progress: {e}")


In [ ]:
# Create Agents
def create_educational_agents(student_id: str, session_id: str):
    progress_memory = ProgressMemory(student_id)
    conv_memory = LangChainConversationMemory(session_id)
    progress_memory.start_session(session_id)

    tutor = AssistantAgent(
        name="Educational_Tutor",
        system_message=AgentConfig.get_tutor_system_message(),
        llm_config=llm_config,
        human_input_mode="NEVER",
        max_consecutive_auto_reply=5
    )

    @retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
    def explain_concept(subject: str, topic: str, difficulty_level: str = "medium", learning_style: str = "visual"):
        context = conv_memory.get_context()
        prompt = f"""Using this conversation history:
{context}

Explain {topic} in {subject} at {difficulty_level} difficulty for a high-school student, using a {learning_style} learning style. Provide:
1. Step-by-step breakdown
2. One example
3. One practice question"""
        response = tutor.generate_reply([{"content": prompt, "role": "user"}])
        conv_memory.add_message("tutor", response, {"subject": subject, "topic": topic})
        progress_memory.update_progress(subject, topic, 0.6, True)
        progress_memory.update_session(session_id, subject, topic, question_asked=True)
        return response

    @retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
    def create_practice_problems(subject: str, topic: str, count: int = 3, difficulty: str = "medium") -> Dict[str, Any]:
        context = conv_memory.get_context()
        prompt = f"""Using this conversation history:
{context}

Generate {count} practice problems for {topic} in {subject} at {difficulty} difficulty. Include:
- One factoring problem
- One quadratic formula problem
- One word problem
Return as JSON with keys: question, correct_answer, solution."""
        response = tutor.generate_reply([{"content": prompt, "role": "user"}])
        json_start = response.find('{')
        json_end = response.rfind('}') + 1
        problems = json.loads(response[json_start:json_end]) if json_start != -1 else {"error": "Invalid response"}
        conv_memory.add_message("tutor", json.dumps(problems, indent=2), {"subject": subject, "topic": topic})
        progress_memory.update_session(session_id, subject, topic, question_asked=True)
        return problems

    tutor.register_function({
        "explain_concept": explain_concept,
        "create_practice_problems": create_practice_problems
    })

    student = UserProxyAgent(
        name="Student_Learner",
        system_message=AgentConfig.get_student_system_message(),
        human_input_mode="NEVER",
        max_consecutive_auto_reply=3,
        code_execution_config=False
    )

    progress_tracker = AssistantAgent(
        name="Progress_Tracker",
        system_message=AgentConfig.get_progress_tracker_system_message(),
        llm_config=llm_config,
        human_input_mode="NEVER"
    )

    @retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
    def generate_progress_report():
        report = progress_memory.get_progress_report()
        if not report["subjects"] and not report["sessions"]:
            return {
                "summary": "No progress data available yet. Try solving some problems!",
                "strengths": [],
                "areas_for_improvement": [],
                "recommendations": ["Solve practice problems", "Ask clarifying questions"],
                "timestamp": datetime.now().isoformat()
            }
        prompt = f"""Generate a student-friendly JSON progress report based on:
{json.dumps(report, indent=2)}
Include:
- Summary of progress
- Strengths (skill_level > 0.7)
- Areas for improvement (skill_level < 0.6)
- Recommendations"""
        response = progress_tracker.generate_reply([{"content": prompt, "role": "user"}])
        json_start = response.find('{')
        json_end = response.rfind('}') + 1
        return json.loads(response[json_start:json_end]) if json_start != -1 else {"error": "Invalid report"}

    progress_tracker.register_function({"generate_progress_report": generate_progress_report})

    def custom_speaker_selection(last_speaker, groupchat):
        agents = groupchat.agents
        last_message = groupchat.messages[-1]["content"].strip() if groupchat.messages else ""
        if last_speaker == student and not last_message:
            return tutor  # Skip empty student responses
        return agents[(agents.index(last_speaker) + 1) % len(agents)]

    groupchat = GroupChat(
        agents=[student, tutor, progress_tracker],
        messages=[],
        max_round=20,
        speaker_selection_method=custom_speaker_selection
    )

    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config=llm_config,
        name="Education_Manager"
    )

    return {
        "student": student,
        "tutor": tutor,
        "progress_tracker": progress_tracker,
        "manager": manager,
        "progress_memory": progress_memory,
        "conv_memory": conv_memory
    }


In [ ]:


# Main Execution
def main():
    print("=== Starting Educational Session ===")
    student_id = "test_student_001"
    session_id = str(uuid.uuid4())

    try:
        agents = create_educational_agents(student_id, session_id)
        student = agents["student"]
        manager = agents["manager"]
        progress_tracker = agents["progress_tracker"]
        progress_memory = agents["progress_memory"]
        conv_memory = agents["conv_memory"]

        # Explain quadratic equations
        print("\n=== Explaining Quadratic Equations ===")
        student.initiate_chat(
            manager,
            message="Please explain quadratic equations to me at a medium difficulty level with visual learning style."
        )
        conv_memory.add_message("student", "Asked about quadratic equations", {"subject": "Mathematics", "topic": "Quadratic Equations"})
        progress_memory.update_progress("Mathematics", "Quadratic Equations", 0.6, True)
        progress_memory.update_session(session_id, "Mathematics", "Quadratic Equations", question_asked=True)

        # Real-world example
        print("\n=== Real-World Example ===")
        student.send(
            message="Can you show a real-world example of quadratic equations?",
            recipient=manager
        )
        conv_memory.add_message("student", "Requested real-world example", {"subject": "Mathematics", "topic": "Quadratic Equations"})
        progress_memory.update_progress("Mathematics", "Quadratic Equations", 0.7, True)
        progress_memory.update_session(session_id, "Mathematics", "Quadratic Equations", question_asked=True)

        # Practice problems
        print("\n=== Creating Practice Problems ===")
        student.send(
            message="Could you create 3 practice problems on quadratic equations at medium difficulty?",
            recipient=manager
        )
        conv_memory.add_message("student", "Requested practice problems", {"subject": "Mathematics", "topic": "Quadratic Equations"})
        progress_memory.update_session(session_id, "Mathematics", "Quadratic Equations", question_asked=True)

        # Student attempts problem
        print("\n=== Student Attempting Problem ===")
        student.send(
            message="For x^2 - 7x + 10 = 0, I tried factoring. I got x = 2 and x = 5. Is this correct?",
            recipient=manager
        )
        conv_memory.add_message("student", "Attempted factoring x^2 - 7x + 10", {"subject": "Mathematics", "topic": "Quadratic Equations"})
        progress_memory.update_progress("Mathematics", "Quadratic Equations", 0.8, True)
        progress_memory.update_session(session_id, "Mathematics", "Quadratic Equations", question_asked=True, correct_answer=True)

        # Generate progress report
        print("\n=== Generating Progress Report ===")
        response = student.initiate_chat(
            progress_tracker,
            message="Please generate a progress report."
        )
        report = None
        for msg in response.chat_history:
            content = msg.get("content", "")
            if isinstance(content, str) and "summary" in content:
                json_start = content.find('{')
                json_end = content.rfind('}') + 1
                if json_start != -1 and json_end != 0:
                    report = json.loads(content[json_start:json_end])
                    break
        if report:
            print(json.dumps(report, indent=2))
        else:
            print("Failed to generate progress report.")

        # Session statistics
        print("\n=== Session Statistics ===")
        final_report = progress_memory.get_progress_report()
        print(f"Total Sessions: {len(final_report['sessions'])}")
        print(f"Total Questions: {sum(s['questions_asked'] for s in final_report['sessions'])}")
        print(f"Subjects Covered: {[s for s in final_report['subjects'].keys()]}")

    except Exception as e:
        logger.error(f"Error in main: {e}")
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

    print("\n=== Educational Session Completed ===")

In [ ]:


if __name__ == "__main__":
    os.environ["AUTOGEN_USE_DOCKER"] = "False"
    main()

=== Starting Educational Session ===
2025-06-04 23:07:02,485 - ERROR - Failed to load progress: LearningProgress.__init__() got an unexpected keyword argument 'confidence_level'


C:\Users\dhanu\AppData\Local\Temp\ipykernel_35320\188603150.py:25: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(return_messages=True)



=== Explaining Quadratic Equations ===
Student_Learner (to Education_Manager):

Please explain quadratic equations to me at a medium difficulty level with visual learning style.

--------------------------------------------------------------------------------

Next speaker: Educational_Tutor

2025-06-04 23:07:12,743 - INFO - HTTP Request: POST https://idkrag.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
Educational_Tutor (to Education_Manager):

Absolutely! Let’s dive into quadratic equations with clear explanations, visual aids, and examples. Quadratic equations are a fundamental part of algebra and appear often in math, physics, engineering, and even economics! 

---

### **What is a Quadratic Equation?**
A quadratic equation is any equation that can be written in the form:

**ax² + bx + c = 0**

Where:
- **a, b, c** are constants (numbers),
- **x** is the variable (unknown),
- The highest power of **x** is **2** (making it "qua

=== Welcome to the Educational Session ===

Available Actions:
1. Explain a concept (e.g., 'Explain quadratic equations, medium difficulty, visual style')
2. Request practice problems (e.g., 'Create 3 problems on quadratic equations, medium difficulty')
3. Attempt a problem (e.g., 'For x^2 - 7x + 10 = 0, I got x = 2, x = 5. Is this correct?')
4. Generate progress report
5. Exit

What would you like to do? (Enter number 1-5)
Invalid action. Please choose 1-5.

What would you like to do? (Enter number 1-5)
Enter practice problems query (e.g., 'Create 3 problems on quadratic equations, medium difficulty'):
Student_Learner (to Education_Manager):

Can you show a real-world example of quadratic equations?

--------------------------------------------------------------------------------
2025-06-04 23:31:56,740 - INFO - Updated session 4a7d0147-99ed-470b-9706-828d2fbd3caf: subject=Mathematics, questions_asked=1

What would you like to do? (Enter number 1-5)
Invalid action. Please choose 1-5.
